# Revise the Income Map Top Ridership Function

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

hunter1 = pd.read_csv("datasets/hunter/MTA_Subway_Origin-Destination_2021_Hunter_Origin.csv")
hunter2 = pd.read_csv("datasets/hunter/MTA_Subway_Origin-Destination_2022_Hunter_Origin.csv")
hunter3 = pd.read_csv("datasets/hunter/MTA_Subway_Origin-Destination_2023_Hunter_Origin.csv")
hunter4 = pd.read_csv("datasets/hunter/MTA_Subway_Origin-Destination_2024_Hunter_Origin.csv")
hunter_total = pd.concat([hunter1, hunter2, hunter3, hunter4])

ccny1 = pd.read_csv("datasets/ccny/MTA_Subway_Origin-Destination_2021_CCNY_Origin.csv")
ccny2 = pd.read_csv("datasets/ccny/MTA_Subway_Origin-Destination_2022_CCNY_Origin.csv")
ccny3 = pd.read_csv("datasets/ccny/MTA_Subway_Origin-Destination_2023_CCNY_Origin.csv")
ccny4 = pd.read_csv("datasets/ccny/MTA_Subway_Origin-Destination_2024_CCNY_Origin.csv")
ccny_total = pd.concat([ccny1, ccny2, ccny3, ccny4])

medgar1 = pd.read_csv("datasets/medgar/MTA_Subway_Origin-Destination_2021_Medgar_Origin.csv")
medgar2 = pd.read_csv("datasets/medgar/MTA_Subway_Origin-Destination_2022_Medgar_Origin.csv")
medgar3 = pd.read_csv("datasets/medgar/MTA_Subway_Origin-Destination_2023_Medgar_Origin.csv")
medgar4 = pd.read_csv("datasets/medgar/MTA_Subway_Origin-Destination_2024_Medgar_Origin.csv")
medgar_total = pd.concat([medgar1, medgar2, medgar3, medgar4])

columbia1 = pd.read_csv("datasets/columbia/MTA_Subway_Origin-Destination_2021_Columbia_Origin.csv")
columbia2 = pd.read_csv("datasets/columbia/MTA_Subway_Origin-Destination_2022_Columbia_Origin.csv")
columbia3 = pd.read_csv("datasets/columbia/MTA_Subway_Origin-Destination_2023_Columbia_Origin.csv")
columbia4 = pd.read_csv("datasets/columbia/MTA_Subway_Origin-Destination_2024_Columbia_Origin.csv")
columbia_total = pd.concat([columbia1, columbia2, columbia3, columbia4])

nyu1 = pd.read_csv("datasets/nyu/MTA_Subway_Origin-Destination_2021_NYU_Origin.csv")
nyu2 = pd.read_csv("datasets/nyu/MTA_Subway_Origin-Destination_2022_NYU_Origin.csv")
nyu3 = pd.read_csv("datasets/nyu/MTA_Subway_Origin-Destination_2023_NYU_Origin.csv")
nyu4 = pd.read_csv("datasets/nyu/MTA_Subway_Origin-Destination_2024_NYU_Origin.csv")
nyu_total = pd.concat([nyu1, nyu2, nyu3, nyu4])

In [2]:
import folium
import geopandas as gpd
import pandas as pd
import json

# Load NYC GeoJSON (Ensure it contains ZIP codes)
geojson_file = "datasets/nyc_zipcode_geodata/nyc-zip-code-tabulation-areas-polygons.geojson"
gdf = gpd.read_file(geojson_file)

# Load ZIP Code Latitude/Longitude Data
zip_coords = pd.read_csv("datasets/nyc_zipcode_geodata/uszipcodes_geodata.csv")

# Load Income Data
income_data = pd.read_csv("datasets/nyc_median_income_zipcode.csv")

# Load MTA Subway Stations
mta_stations = pd.read_csv("datasets/MTA_Subway_Stations_and_Complexes_20250225.csv")

# Convert ZIP codes to strings for proper merging
gdf["postalCode"] = gdf["postalCode"].astype(str)
zip_coords["ZIP"] = zip_coords["ZIP"].astype(str)
income_data["zipcode"] = income_data["zipcode"].astype(str)

# Merge GeoDataFrame with ZIP code coordinates and income data
gdf = gdf.merge(zip_coords, left_on="postalCode", right_on="ZIP", how="left")
gdf = gdf.merge(income_data, left_on="postalCode", right_on="zipcode", how="left")

# Convert merged GeoDataFrame to JSON
geojson_data = json.loads(gdf.to_json())

# Define color scale based on income
def get_income_color(income):
    if pd.isna(income):
        return "gray"
    elif income < 50000:
        return "red"
    elif income < 100000:
        return "orange"
    elif income < 150000:
        return "yellow"
    elif income < 200000:
        return "lightgreen"
    else:
        return "green"
    
# Function to get the top station origins
def top_station_destinations(ridership_df, top_n=5):
    top_destinations = ridership_df.groupby(["Destination Station Complex Name", "Destination Station Complex ID", "Destination Latitude", "Destination Longitude"])["Estimated Average Ridership"].sum().sort_values(ascending=False).head(top_n)
    return top_destinations


def top_destination_income_map(ridership_df, top_n, start_time, end_time, day_of_week, month, year):

    # Filter the ridership data for the specified time range
    ridership_df = ridership_df[(ridership_df["Hour of Day"] >= start_time) & (ridership_df["Hour of Day"] <= end_time)]
    ridership_df = ridership_df[ridership_df["Day of Week"] == day_of_week]
    ridership_df = ridership_df[ridership_df["Month"] == month]
    ridership_df = ridership_df[ridership_df["Year"] == year]

    # Create a folium map centered on NYC
    m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

    # Add GeoJSON layer with income-based coloring and tooltips
    folium.GeoJson(
        geojson_data,
        name="NYC Neighborhoods",
        style_function=lambda x: {
            "fillColor": get_income_color(x["properties"].get("income_household_median", None)),
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.6,
        },
        tooltip=folium.GeoJsonTooltip(
            fields=["postalCode", "income_household_median"],
            aliases=["ZIP Code", "Median Household Income"],
            localize=True,
        ),
    ).add_to(m)

    # Get the top destinations
    top_destinations_df = top_station_destinations(ridership_df, top_n)

    # Ensure top_destinations_df is a DataFrame
    if isinstance(top_destinations_df, pd.Series):
        top_destinations_df = top_destinations_df.reset_index()

    # Normalize ridership values for marker sizing
    max_ridership = top_destinations_df["Estimated Average Ridership"].max()
    min_radius = 5
    max_radius = 15

    # Add MTA Subway Stations as circle markers with dynamic radius
    for _, row in top_destinations_df.iterrows():
        if pd.notna(row["Destination Latitude"]) and pd.notna(row["Destination Longitude"]):
            ridership = row["Estimated Average Ridership"]
            radius = min_radius + (ridership / max_ridership) * (max_radius - min_radius)

            folium.CircleMarker(
                location=[row["Destination Latitude"], row["Destination Longitude"]],
                radius=radius,
                color="blue",
                fill=True,
                fill_color="blue",
                fill_opacity=0.7,
                popup=folium.Popup(
                    f"Station: {row['Destination Station Complex Name']}<br>"
                    f"Station ID: {row['Destination Station Complex ID']}<br>"
                    f"Ridership: {ridership:,.0f}",
                    max_width=300,
                ),
                tooltip=row["Destination Station Complex Name"],
            ).add_to(m)

    # Add origin station for reference
    origin_df = ridership_df.iloc[0]
    folium.CircleMarker(
                location=[origin_df["Origin Latitude"], origin_df["Origin Longitude"]],
                radius=10,
                color="red",
                fill=True,
                fill_color="red",
                fill_opacity=0.7,
                popup=folium.Popup(
                    f"Station: {origin_df['Origin Station Complex Name']}<br>"
                    f"Station ID: {origin_df['Origin Station Complex ID']}<br>",
                    max_width=300,
                ),
                tooltip=origin_df["Origin Station Complex Name"],
            ).add_to(m)
    
    return m

Code to save folium map html file as a png image: use this to create timelapses of interest

In [ ]:
map = top_destination_income_map(hunter_total, 15, start_time=18, end_time=19, day_of_week="Tuesday", month=1, year=2021)


In [7]:
import io
from PIL import Image

img_data = map._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')